In [105]:
import pandas as pd
import numpy as np
import requests
import json
import pickle
import os

from sklearn.neighbors import NearestNeighbors
from sklearn.base import BaseEstimator, TransformerMixin

from flask import Blueprint, request, jsonify, render_template, Flask
from flask_cors import CORS, cross_origin


In [106]:
class My_encoder(BaseEstimator, TransformerMixin):
   
    def __init__(self,drop = 'first',sparse=False):
        self.encoder = OneHotEncoder(drop = drop,sparse = sparse)
        self.features_to_encode = []
        self.columns = []
    
    def fit(self,X_train,features_to_encode):
        
        data = X_train.copy()
        self.features_to_encode = features_to_encode
        data_to_encode = data[self.features_to_encode]
        self.columns = pd.get_dummies(data_to_encode,drop_first = True).columns
        self.encoder.fit(data_to_encode)
        return self.encoder
    
    def transform(self,X_test):
        
        data = X_test.copy()
        data.reset_index(drop = True,inplace =True)
        data_to_encode = data[self.features_to_encode]
        data_left = data.drop(self.features_to_encode,axis = 1)
        
        data_encoded = pd.DataFrame(self.encoder.transform(data_to_encode),columns = self.columns)
        
        return pd.concat([data_left,data_encoded],axis = 1)

In [ ]:
def safe_paths():
    '''
    Function helper to define path of pickled model and preprocessed dataframes.
    '''
    
    model_path = os.path.join('model_pickle.pkl')
    train_path = os.path.join('df_encoded_for_train.pkl')
    ref_path = os.path.join('df_encoded_for_ref.pkl')
    
    return (model_path, train_path, ref_path)



def load_models():
    '''
    Helper function to load pickled model and preprocessed dataframes
    '''
    model_path, train_path, ref_path = safe_paths()
    
    pickled_model = pickle.load(open(model_path, 'rb'))
    pickled_train = pickle.load(open(train_path, 'rb'))
    pickled_ref = pickle.load(open(ref_path, 'rb'))
        
    return (pickled_model, pickled_train, pickled_ref)  



############################################################
'Saving loaded pickled files in global memory'

PICKLED_MODEL, PICKLED_TRAIN, PICKLED_REF = load_models()
#############################################################


def recommend_track_id(track_id = '2YegxR5As7BeQuVp2U6pek'):
    '''
    Helper function that takes in atrack_id and recommends 14 track_ids of similar songs
    '''
    
    song_num = PICKLED_REF[PICKLED_REF['track_id']==track_id].song_num.to_numpy()[0]
    distance, neighbors = PICKLED_MODEL.kneighbors(np.array([PICKLED_TRAIN.values[song_num]]))
    
    song_list = []
    for item in neighbors[0][1:]: # this way excludes itself
        row = PICKLED_REF.iloc[item]
        song_list.append((row.track_id))  # print list of track id
      
    return song_list


def recommend_json(track_id='2YegxR5As7BeQuVp2U6pek'):
    '''
    Helper function the returns track_ids in json format.
    '''
    
    rec_list = recommend_track_id(track_id)
    rec_df = pd.DataFrame(rec_list, columns=['rec_track_id'])
    rec_json = rec_df.to_json()
    
    return rec_json
    
    
    

In [116]:
recommend_track_id('4FCb4CUbFCMNRkI6lYc1zI')

['0DJev1kGOCzaY8Ae3oQwH0',
 '3t7wojJJqpSNCASv5wOe5I',
 '3e0qSxu1fNPzDlwqWN3cBD',
 '2bQ6JedH8i3lrYLD0Zw57k',
 '44lusH2vzc4sRAwsooRhPh',
 '6ZOBP3NvffbU4SZcrnt1k6',
 '5sZ3xD192Bksbg6uBvd8Ig',
 '6ZzYETKetIfNUsZUb23jgG',
 '5ROshySAFh5odQX66q0anm',
 '0q6LuUqGLUiCPP1cbdwFs3',
 '0W9Xvd4Qx1aZPxEi94vgRY',
 '3GZD6HmiNUhxXYf8Gch723',
 '5PntSbMHC1ud6Vvl8x56qd',
 '4e9eGQYsOiBcftrWXwsVco']

In [117]:
recommend_json()

'{"rec_track_id":{"0":"6jvvpPJQJy5rMOEkLlADl6","1":"17h2obBWaYI5UaRMtwnazd","2":"645hAlwdOj1cx9dbf5Yu8A","3":"3XnknFTGnFwK2Ffmgg1BH3","4":"74tQaH4CxM5Rs9BzerEbHr","5":"5o54O3MOnMvcPTmkguYkwg","6":"4CMrdHWqic0usIZfTrKoI3","7":"4CbKVDZkYKdv69I4bCaKUq","8":"0c3gqQj20DX64pnVC4gLGR","9":"3rjM7GhxdVq1YySsHBs21i","10":"2z4pcBLQXF2BXKFvd0BuB6","11":"2nLtzopw4rPReszdYBJU6h","12":"3fMKnwiByu9yfeD5ISn9Et","13":"0bxmVPKnEopTyuMMkaTvUb"}}'